<a href="https://colab.research.google.com/github/eightuponatime/Platonus-Parser/blob/main/bleurt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**клонирование репозитория**

In [ ]:
!git clone https://github.com/google-research/bleurt.git

Cloning into 'bleurt'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 134 (delta 0), reused 17 (delta 0), pack-reused 116
Receiving objects: 100% (134/134), 31.28 MiB | 24.90 MiB/s, done.
Resolving deltas: 100% (49/49), done.


**переход в папку установщика**

In [ ]:
%cd bleurt

/content/bleurt


**вызов setup.py**

In [ ]:
!pip install .

Processing /content/bleurt
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456763 sha256=158dfd4b9ee6d96ad8c3a47e05347afce8928983bd3bd01a94e0a4e1156a242d
  Stored in directory: /tmp/pip-ephem-wheel-cache-qkbuy90p/wheels/92/4f/fb/afa555fa27aa9e2c7958df797a62cc4e74f0f459cec9c4fa7c
Successfully built BLEURT


**пара тестов для bleurt**

In [ ]:
from bleurt import score

def candidate_score(reference, candidate):

  checkpoint = "bleurt/test_checkpoint"
  references = [reference]
  candidates = [candidate]

  scorer = score.BleurtScorer(checkpoint)
  scores = scorer.score(references=references, candidates=candidates)
  assert isinstance(scores, list) and len(scores) == 1

  return scores

ref = "твоя моя давать задания"
candidate_list = [
  "Ты должен дать мне свои задания", #true
  "Я даю тебе свои задания",
  "Я даю тебе мои задания",
  "Я должен дать тебе мои задания",
  "Ты должен дать мне мои задания",
]

print("========================")
print(f"REFERENCE : {ref}")
for candidate in candidate_list:
  print(f"{candidate} : {candidate_score(ref, candidate)}")

ref = "я грзин , я ни говорить по рузки"
candidate_list = [
	"Я грузин, я не говорю по-русски.", #true
	"Я грузин, я ни говорить по-русски.",
	"Я грузин, я ни говорю по-русски.",
	"Я грузин, я не говорю по-русски.",
	"Я грузин, я ни говорю по-русски.",
]

print("========================")
print(f"REFERENCE : {ref}")
for candidate in candidate_list:
  print(f"{candidate} : {candidate_score(ref, candidate)}")


REFERENCE : твоя моя давать задания
Ты должен дать мне свои задания : [0.5622445940971375]
Я даю тебе свои задания : [0.31740158796310425]
Я даю тебе мои задания : [0.2844545245170593]
Я должен дать тебе мои задания : [0.374971479177475]
Ты должен дать мне мои задания : [0.4611471891403198]
REFERENCE : я грзин , я ни говорить по рузки
Я грузин, я не говорю по-русски. : [0.6609145998954773]
Я грузин, я ни говорить по-русски. : [0.8238028287887573]
Я грузин, я ни говорю по-русски. : [0.6871737837791443]
Я грузин, я не говорю по-русски. : [0.6609145998954773]
Я грузин, я ни говорю по-русски. : [0.6871737837791443]


**проверка многопоточности :)**
на cpu нет смысла.
на gpu быстрее.


In [ ]:
from bleurt import score
import concurrent.futures
import time

def candidate_score(reference, candidate):

  checkpoint = "bleurt/test_checkpoint"
  references = [reference]
  candidates = [candidate]

  scorer = score.BleurtScorer(checkpoint)
  scores = scorer.score(references=references, candidates=candidates)
  assert isinstance(scores, list) and len(scores) == 1

  return f"{candidate} : {scores}"

def threaded_execution(ref, candidate_list, max_workers=5):
  print("running threaded:")
  threaded_start = time.time()
  with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = []
    for candidate in candidate_list:
      futures.append(executor.submit(candidate_score, reference=ref,
                                    candidate=candidate))
    for future in concurrent.futures.as_completed(futures):
      print(future.result())

  print("threaded time:", time.time() - threaded_start)

ref = "твоя моя давать задания"
candidate_list = [
  "Ты должен дать мне свои задания", #true
  "Я даю тебе свои задания",
  "Я даю тебе мои задания",
  "Я должен дать тебе мои задания",
  "Ты должен дать мне мои задания",
]

print("==========================")
print(f"REFERENCE: {ref}")
threaded_execution(ref, candidate_list)

ref = "я грзин , я ни говорить по рузки"
candidate_list = [
	"Я грузин, я не говорю по-русски.", #true
	"Я грузин, я ни говорить по-русски.",
	"Я грузин, я ни говорю по-русски.",
	"Я грузин, я не говорю по-русски.",
	"Я грузин, я ни говорю по-русски.",
]

print("==========================")
print(f"REFERENCE: {ref}")
threaded_execution(ref, candidate_list)


REFERENCE: твоя моя давать задания
running threaded:
Ты должен дать мне свои задания : [0.5622445940971375]
Ты должен дать мне мои задания : [0.4611471891403198]
Я даю тебе свои задания : [0.31740158796310425]
Я должен дать тебе мои задания : [0.374971479177475]
Я даю тебе мои задания : [0.2844545245170593]
threaded time: 4.271783828735352
REFERENCE: я грзин , я ни говорить по рузки
running threaded:
Я грузин, я не говорю по-русски. : [0.6609145998954773]
Я грузин, я ни говорю по-русски. : [0.6871737837791443]
Я грузин, я не говорю по-русски. : [0.6609145998954773]
Я грузин, я ни говорю по-русски. : [0.6871737837791443]
Я грузин, я ни говорить по-русски. : [0.8238028287887573]
threaded time: 4.719408750534058
